### Import Required Libraries and Set Up Environment Variables

In [92]:
# Import Dependencies for the script
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [93]:
# Set environment variables from the .env in the local environment
load_dotenv()
nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API
#### Build url and Filter


In [103]:
# # Build URL
# build_url = f"{url}api-key={nyt_api_key}&q={filter_query}&sort={sort}&begin_date={begin_date}&end_date={end_date}&fl={field_list}"  
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [123]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(2):  # NYT API limits results to 10 per page, 20 pages = 200 results
    
    # create query with a page number
    # API results show 10 articles at a time
    page_query = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for responses in response["response"]["docs"]:
            reviews_list.append(responses)
        # Print the page that was just retrieved
        print(f"Checked page {page}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"Page empty, no results found")
        break

        

Checked page 0
[{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}], 'pub_date': '2023-05-25T11:00:03+0000', 'byline': {'original': 'By Jeannette Catsoulis', 'person': [{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}], 'organization': None}, '

In [102]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/31/movies/reality-review.html",
        "snippet": "A new docudrama starring Sydney Sweeney as Reality Winner is gripping, even as it strips a true story of its political context.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Reality\u2019 Review: An Unusual Suspect",
            "kicker": "Critic\u2019s Pick",
            "content_kicker": null,
            "print_headline": "Examining A Most Unusual Suspect",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "Reality (2023) (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "

In [71]:
# # API results show 10 articles at a time
#     # Number of articles per page
#     articles_per_page = 10 
#     api_key = nyt_api_key # Your API key from the NY Times API
# # Construct the query URL with placeholders for page number and limit
#     #query = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key={api_key}&q=love&fq=section_name&page={page_number}&limit={articles_per_page}"
#     query_page = f"{query_url}&page={page_number}"
#     # Make a "GET" request and retrieve the JSON
#     # url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
#     # Make a "GET" request and retrieve the JSON response
#     response = requests.get(query_page)
#     response_json = response.json()
#     # Check if the request was successful (status code 200)
#     try:  
    # # Check if the request was successful (status code 200)
    #     if response.status_code == 200:
    #         # Parse the JSON response
    #         json_data = response.json()
    #         # Now you can work with the JSON data
    #         # Try and save the reviews to the reviews_list
    #         for review in json_data["response"]["docs"]:
    #             reviews_list.append(review)
    #         # Print the page that was just retrieved
    #         print("Retrieved page:", page_number)
    #         # Print the page number that had no results then break from the loop
    #         # if not json_data["response"]["docs"]:
    #         #     print("No results for page number:", page_number)
    #     else:
    #             # If the request was not successful, print an error message
    #         print("Error:", response.status_code)
    #     # Add a twelve-second interval between queries to stay within API query limits
    #     time.sleep(12)
    # except:
    #     print("Error in retrieving page", page_number)

In [132]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_normalized_df = pd.json_normalize(reviews_list)
reviews_list_normalized_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'fir

In [73]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [74]:
# Extract the title from the "headline.main" column and
# save it to a new column "title" in a normalized DataFrame
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_list_normalized_df['title'] = reviews_list_normalized_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_list_normalized_df
print(reviews_list_normalized_df['title'])


0                            The Attachment Diaries
1     What’s Love Got to Do With It?’ Probably a Lo
2                              You Can Live Forever
3                         A Tourist’s Guide to Love
4                           Other People’s Children
5                                    One True Loves
6                    The Lost Weekend: A Love Story
7                                A Thousand and One
8                                Your Place or Mine
9                      Love in the Time of Fentanyl
10                             Pamela, a Love Story
11                                 In From the Side
12                                       After Love
13                                         Alcarràs
14                                   Nelly & Nadine
15                          Lady Chatterley’s Lover
16                           The Sound of Christmas
17                                   The Inspection
18                                    Bones and All
19          

In [75]:
print(reviews_list_normalized_df.columns)

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title'],
      dtype='object')


In [76]:
# Use the supplied extract_keywords function to convert the "keywords" column 
# from a list of dictionaries to strings using the apply() method.
# Apply the extract_keywords function to convert the "keywords" column to strings
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

In [77]:
# Create a list called titles from the "title" column using to_list(). 
# These titles will be used in the query for The Movie Database
titles_list = reviews_list_normalized_df["title"].squeeze().to_list()
print(titles_list)

['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman']


### Access The Movie Database API

##### Prepare the Movie Query

In [78]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

##### Create a Movie List

In [91]:
tmdb_movies_list = []

# Initialize request counter
request_counter = 1

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(12)  
        print("Sleeping for 12 seconds")
    # Increment request counter
    request_counter += 1

    # Convert title to string explicitly if needed (unnecessary here)
    # title = str(title)

    # Perform a GET request
    response = requests.get(url + title + tmdb_key_string).json()
    print(response)

    # Include try-except block for error handling
try:
    # Check if there are any results in the response
    if response["results"]:
        # Get the movie id from the first result
        movie_id = response["results"][0]["id"]
        print(movie_id)
        # Make a GET request for full movie details
        full_movie_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}").json()

        # Extract necessary data
        genres = [genre["name"] for genre in full_movie_response["genres"]]
        spoken_languages = [language["english_name"] for language in full_movie_response["spoken_languages"]]
        production_countries = [country["name"] for country in full_movie_response["production_countries"]]

        # Append data to tmdb_movies_list
        movie_data = {
            "title": title,
            "genre_names": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries,
        }
        tmdb_movies_list.append(movie_data)

        # Print out success message
        print(f"Title found: {title}")
    else:
        print(f"No results found for: {title}")

except IndexError:
    print(f"No results found for: {title}")


# Preview the first five results
print(json.dumps(tmdb_movies_list[:5], indent=4))

# Convert tmdb_movies_list to a Pandas DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)

# # Loop through the pages
# for page_number in range(2):
#     # Construct the API request URL for the current page
#     api_url = f"{url}page={page_number + 1}{tmdb_key_string}"
    
#     # Make the request to the Movie Database API
#     response = requests.get(api_url)
#     # Check if the request was successful (status code 200)
#     # Check if the request counter is a multiple of 50
#     if request_counter % 50 == 0:
    
#     # Print a message to indicate that the script is pausing
#     # Check if the request was successful (status code 200)
#     if response.status_code == 200:
#         # Parse the JSON response
    #     response_json = response.json()
    #     # Process the response and extract relevant data
    #     # For example, you might extract movie titles, release dates, etc.
    #     # Append the extracted data to the results list
    #     results.extend(response_json['results'])
    # else:
    #     # If the request was not successful, print an error message
    #     print("Error:", response.status_code)
    
    # # Make the API request here
    # # Include a try clause to search for the full movie details.
    # # Your code for making API requests and processing responses goes here
    # # Build URL
    # query_url = (
    # f"{url}api-key={tmdb_api_key}&begin_date={begin_date}&end_date={end_date}"
    # + f'&fq={filter_query}&sort={sort}&fl={field_list}')
    
    # # Example: Print the current page number
    # print("Processing page:", page_number)
# # Make the request and process the response
# # Define the query parameter for the movie search
# # Construct the full URL for the search request
# full_url = f"{url}api_key={tmdb_api_key}&query={query}"
# try:
#     # Make the API request to search for movies
#     response = requests.get(full_url)
#     # Check if the request was successful (status code 200)
#     if response.status_code == 200:
#         # Parse the JSON respons
#         search_results = response.json()
#         # Process the search results and extract relevant information
#         for movie in search_results["results"]:
#             # For each movie in the search results, you might want to fetch additional details
#             # Make additional requests to fetch full movie details, cast, etc.
#             movie_id = movie["id"]
#             # Example: Make a request to fetch full movie details
#             movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
# #             movie_details_response = requests.get(movie_details_url)
#             if movie_details_response.status_code == 200:
#                 movie_details = movie_details_response.json()
#                 # Process and extract relevant information from movie_details
#         # Your code for processing search results and extracting relevant information goes here
#     else:
#         # If the request was not successful, print an error message
#         print("Error:", response.status_code)
# except Exception as e:
#     # If an exception occurs during the request, print the error message
#     print("Error:", e)
#     # Perform a "GET" request for The Movie Database
#     api_url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}"
#     response = requests.get(api_url)
#     display(response.json())
#     # Check if the request was successful (status code 200) and Process the response as needed
#     # print(response.json())
#     # Include a try clause to search for the full movie details and extract the relevant data
# try:
    # Make the API request to search for movies
    # Use the except clause to print out a statement or error if a movie is not found.
#     full_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
#     full_movie_response = requests.get(full_movie_url)
#  # Check if the request was successful
#     if full_movie_response.status_code == 200:
        # Parse the JSON response
#         full_movie_details = full_movie_response.json()
#         # Process and extract relevant information from full movie details
#         # Example: Print movie details
#         print(full_movie_details)
#     else:
#         # If the request was not successful, print an error message
#         print("Error:", full_movie_response.status_code)
# except Exception as e:
#     # If an exception occurs during the request, print the error message
#     print("Error:", e)
#     # Use the except clause to print out a statement if a movie is not found
#     print("Movie not found.")
#         # Get movie id
#     movie_id = response.json()["data"]["movie_id"]
#         # Make a request for a the full movie details
#     full_movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
#         # Execute "GET" request with url
#     full_movie_response = requests.get(full_movie_url)
 
#         # Extract the spoken_languages' English name into a list
#     spoken_languages = [language["english_name"] for language in full_movie_response.json()["spoken_languages"]]
#         # Extract the production_countries' name into a list
#     production_countries = [country["name"] for country in full_movie_response.json()["production_countries"]]
#         # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
        
#         # Print out the title that was found
#     print(f"Title found: {title}")




{'page': 1, 'results': [{'adult': False, 'backdrop_path': None, 'genre_ids': [18, 9648, 53, 27], 'id': 743040, 'original_language': 'es', 'original_title': 'El apego', 'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.', 'popularity': 1.435, 'poster_path': '/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg', 'release_date': '2021-10-07', 'title': 'The Attachment Diaries', 'video': False, 'vote_average': 3.0, 'vote_count': 3}], 'total_pages': 1, 'total_results': 1}
{'page': 1, 'results': [], 'total_pages': 1, 'total_results': 0}
{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg', 'genre_ids': [18, 10749], 'id': 887580,

In [89]:
for title in titles_list:
    print(title)

The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo
You Can Live Forever
A Tourist’s Guide to Love
Other People’s Children
One True Loves
The Lost Weekend: A Love Story
A Thousand and One
Your Place or Mine
Love in the Time of Fentanyl
Pamela, a Love Story
In From the Side
After Love
Alcarràs
Nelly & Nadine
Lady Chatterley’s Lover
The Sound of Christmas
The Inspection
Bones and All
My Policeman


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


[]


In [ ]:
# Convert the results to a DataFrame
df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [134]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_list_normalized_df, tmdb_movies_df, on="title", how="left")    



KeyError: 'title'

In [133]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
